In [1]:
# Dependencies
import requests
import json
import pandas as pd

In [2]:
#check url response
apikey ="f98921da60b71947056633acb25f5147"
search_url = f"https://developers.zomato.com/api/v2.1/search?entity_id=259&entity_type=city"
print(requests.get(search_url, headers={"user-key" : apikey}))



<Response [200]>


In [15]:
#check data structure
# Retrieving data and converting it into JSON
data = requests.get(search_url, headers={"user-key" : apikey}).json()
#display data structure
data


{'results_found': 9131,
 'results_start': 0,
 'results_shown': 0,
 'restaurants': []}

In [4]:
resto_num = data["results_found"]
print(f"Number of Rest: {resto_num}")

Number of Rest: 17502


In [5]:
#single city 
#print(json.dumps(data, indent=4, sort_keys=True))

rest_id_list = []
rest_name_list =[]
rest_cuis_list = []
rest_local_list = []
rest_city_list = []
rest_rate_list = []
rest_delivery_list = []
rest_ratenum_list = []

resto_num = 20
show_page = 0

citynum =259

search_url = f"https://developers.zomato.com/api/v2.1/search?entity_id={citynum}&entity_type=city&start={show_page}"
data = requests.get(search_url, headers={"user-key" : apikey}).json()
for page in range(5):
    for count in range(resto_num):
        try: 
            rest_id = data["restaurants"][count]["restaurant"]["id"]
            rest_name = data["restaurants"][count]["restaurant"]["name"]
            rest_cuis = data["restaurants"][count]["restaurant"]["cuisines"]
            rest_local = data["restaurants"][count]["restaurant"]["location"]["locality"]
            rest_city = data["restaurants"][count]["restaurant"]["location"]["city"]
            rest_rate_num = data["restaurants"][count]["restaurant"]["all_reviews_count"]
            rest_rate = data["restaurants"][count]["restaurant"]["user_rating"]["aggregate_rating"]
            rest_delivery = data["restaurants"][count]["restaurant"]["highlights"]
            
            #check if current restaurant does delivery 
            theydelivery = False
            for doesdelivery in rest_delivery:
                if doesdelivery == 'Delivery':
                    theydelivery = True
            
            if theydelivery == True:
                rest_delivery = "Yes"
            else:
                rest_delivery = "No"
            
            #append to list
            rest_id_list.append(rest_id)
            rest_name_list.append(rest_name)
            rest_cuis_list.append(rest_cuis)
            rest_local_list.append(rest_local)
            rest_city_list.append(rest_city)
            rest_rate_list.append(rest_rate)
            rest_ratenum_list.append(rest_rate_num)
            rest_delivery_list.append(rest_delivery)
            
            #go to next page
            if count == 19:
                show_page = show_page + 20
                search_url = f"https://developers.zomato.com/api/v2.1/search?entity_id={citynum}&entity_type=city&start={show_page}"
                data = requests.get(search_url, headers={"user-key" : apikey}).json()
                count = 0
        except:
            print("error")
#check errors when making dataframe
try:            
    restaurants_df = pd.DataFrame({
        "Restaurant ID":rest_id_list,
        "Name":rest_name_list,
        "Cuisine":rest_cuis_list,
        "Locality":rest_local_list,
        "City":rest_city_list,
        "Rating":rest_rate_list,
        "Total Number Ratings": rest_ratenum_list,
        "Delivery":rest_delivery_list
    })
except:
    print(len(rest_id_list))
    print(len(rest_name_list))
    print(len(rest_cuis_list))
    print(len(rest_local_list))
    print(len(rest_city_list))
    print(len(rest_rate_list))
    print(len(rest_review_list))
    print(len(rest_delivery_list))
    

restaurants_df

,Restaurant ID,Name,Cuisine,Locality,City,Rating,Total Number Ratings,Delivery
0,16577492,Chin Chin,"Thai, Asian Fusion",CBD,Melbourne,4.6,3206,No
1,16585905,Tipo 00,Italian,CBD,Melbourne,4.9,721,No
2,16577212,Beatrix,"Coffee and Tea, Bakery",North Melbourne,Melbourne,4.9,384,No
3,16583098,Lune Croissanterie,"Bakery, French",Fitzroy,Melbourne,4.9,753,No
4,16573665,The Hardware Societe,"French, Cafe Food, Coffee and Tea",CBD,Melbourne,4.6,2103,No
...,...,...,...,...,...,...,...,...
95,16574121,Don Don,"Japanese, Asian",CBD,Melbourne,4.6,692,Yes
96,16582908,Mugen Ramen & Bar,"Japanese, Ramen",CBD,Melbourne,4.4,349,No
97,16586156,The Grand Trailer Park Taverna,"Burger, American",CBD,Melbourne,4.4,668,Yes
98,16579874,Two Birds One Stone,"Coffee and Tea, Modern Australian, Cafe Food",South Yarra,Melbourne,4.4,684,No


In [6]:
#save dataframe single city to csv
restaurants_df = restaurants_df.drop_duplicates()
restaurants_df.to_csv("single_restaurant_data.csv", encoding="utf-8", index=False)

In [21]:
#multiple cities
#Perth,Melbourne,Sydney,Brisbane,Adelaide,Gold Coast,Canberra(ACT),Newcastle,Wollongong,Logan City(Village),Geelong
#print(json.dumps(data, indent=4, sort_keys=True))
cityid = [296,259,260,298,297,2555,313,2246,2171,2493,1701]

rest_id_list = []
rest_name_list =[]
rest_cuis_list = []
rest_local_list = []
rest_city_list = []
rest_rate_list = []
rest_delivery_list = []
rest_ratenum_list = []
rest_lat_list = []
rest_long_list = []

resto_num = 20
loop_num = 0
show_page = 0

for citynum in cityid:
    search_url = f"https://developers.zomato.com/api/v2.1/search?entity_id={citynum}&entity_type=city&start={show_page}"
    data = requests.get(search_url, headers={"user-key" : apikey}).json()
    for page in range(5):
        for count in range(resto_num):
            try: 
                rest_id = data["restaurants"][count]["restaurant"]["id"]
                rest_name = data["restaurants"][count]["restaurant"]["name"]
                rest_cuis = data["restaurants"][count]["restaurant"]["cuisines"]
                rest_local = data["restaurants"][count]["restaurant"]["location"]["locality"]
                rest_city = data["restaurants"][count]["restaurant"]["location"]["city"]
                rest_lat = data["restaurants"][count]["restaurant"]["location"]["latitude"]
                rest_long = data["restaurants"][count]["restaurant"]["location"]["longitude"]
                rest_rate_num = data["restaurants"][count]["restaurant"]["all_reviews_count"]
                rest_rate = data["restaurants"][count]["restaurant"]["user_rating"]["aggregate_rating"]
                rest_delivery = data["restaurants"][count]["restaurant"]["highlights"]
                
                #check if current restaurant does delivery 
                theydelivery = False
                for doesdelivery in rest_delivery:
                    if doesdelivery == 'Delivery':
                        theydelivery = True

                if theydelivery == True:
                    rest_delivery = "Yes"
                else:
                    rest_delivery = "No"

                #append to list
                rest_id_list.append(rest_id)
                rest_name_list.append(rest_name)
                rest_cuis_list.append(rest_cuis)
                rest_local_list.append(rest_local)
                rest_city_list.append(rest_city)
                rest_rate_list.append(rest_rate)
                rest_ratenum_list.append(rest_rate_num)
                rest_delivery_list.append(rest_delivery)
                rest_lat_list.append(rest_lat)
                rest_long_list.append(rest_long)
                #go to next page
                if count == 19:
                    show_page = show_page + 20
                    search_url = f"https://developers.zomato.com/api/v2.1/search?entity_id={citynum}&entity_type=city&start={show_page}"
                    data = requests.get(search_url, headers={"user-key" : apikey}).json()
                    count = 0
            except:
                print("error")
    count = 0
    page = 0
    show_page = 0


#check errors when making dataframe
try:            
    restaurants_df = pd.DataFrame({
        "Restaurant ID":rest_id_list,
        "Name":rest_name_list,
        "Cuisine":rest_cuis_list,
        "Locality":rest_local_list,
        "City":rest_city_list,
        "Rating":rest_rate_list,
        "Total Number Ratings": rest_ratenum_list,
        "Delivery":rest_delivery_list,
        "Latitude":rest_lat_list,
        "Longitude":rest_long_list
    })
except:
    print(len(rest_id_list))
    print(len(rest_name_list))
    print(len(rest_cuis_list))
    print(len(rest_local_list))
    print(len(rest_city_list))
    print(len(rest_rate_list))
    print(len(rest_review_list))
    print(len(rest_delivery_list))
restaurants_df = restaurants_df.drop_duplicates()
restaurants_df

,Restaurant ID,Name,Cuisine,Locality,City,Rating,Total Number Ratings,Delivery,Latitude,Longitude
0,16600038,Bib & Tucker,"Modern Australian, Cafe Food",North Fremantle,Perth,4.1,692,No,-32.0284215000,115.7498293000
1,16597645,Mrs. S,"Patisserie, Cafe Food",Maylands,Perth,4.5,643,No,-31.9286800875,115.8919624239
2,18196379,tbsp.,"Sandwich, Coffee and Tea, Cafe Food",Bayswater,Perth,4.7,335,No,-31.9194560000,115.9096643000
3,16598431,Tuck Shop Cafe,"Modern Australian, Cafe Food",Northbridge,Perth,4.8,799,No,-31.9467356499,115.8621080965
4,16599082,Sayers Sister,"Modern Australian, Coffee and Tea",Perth CBD,Perth,4.6,496,No,-31.9404785312,115.8615736663
...,...,...,...,...,...,...,...,...,...,...
1095,16604478,Kohinoor Indian Restaurant,Indian,Geelong CBD,Geelong,3.8,29,No,-38.1479230000,144.3637390000
1096,16606497,The Geelong Boat House,Seafood,Geelong CBD,Geelong,3.2,18,No,-38.1428964000,144.3605468000
1097,16607360,Feed My Mojo,"Coffee and Tea, Fast Food, Cafe Food",Belmont,Geelong,3.9,26,No,-38.1703540000,144.3485380000
1098,18397507,Rook : Sri Lankan Fusion,"Sri Lankan, Cafe Food",Geelong CBD,Geelong,3.8,19,Yes,-38.1479093911,144.3625134230


In [22]:
restaurants_df["City"].value_counts()

Brisbane              101
Perth                 100
Sydney                100
Adelaide              100
Gold Coast            100
Newcastle             100
Wollongong            100
ACT                   100
Melbourne             100
Geelong               100
Eagle Heights           5
Ormeau                  4
Canungra                3
North Tamborine         3
Beaudesert              3
Logan Village           3
Stapylton               3
Advancetown             2
Tamborine               2
Tamborine Mountain      2
Willow Vale             1
Mount Nathan            1
Jimboomba               1
Mount Tamborine         1
Wongawallan             1
Mt Tamborine            1
Springbrook             1
Alberton                1
Beechmont               1
Name: City, dtype: int64

In [23]:
#save dataframe multi city to csv 
restaurants_df = restaurants_df.drop_duplicates()
restaurants_df.to_csv("multi_restaurant_data.csv", encoding="utf-8", index=False)

In [11]:
#check deliveries
restaurants_df["Delivery"].value_counts()

No     273
Yes     27
Name: Delivery, dtype: int64

In [12]:
restaurants_df = restaurants_df.sort_values(by=["Rating"],ascending=False)
restaurants_df

,Restaurant ID,Name,Cuisine,Locality,City,Rating,Total Number Ratings,Delivery
138,17881527,Dexter,"American, BBQ",Preston,Melbourne,4.9,684,No
263,15545439,Manpuku - Chatswood,"Japanese, Ramen",Chatswood,Sydney,4.9,196,No
139,16582574,Hinoki Japanese Pantry,"Japanese, Sushi",Fitzroy,Melbourne,4.9,271,No
229,16559171,Tetsuya's,Japanese,CBD,Sydney,4.9,330,No
232,16558798,Quay,Modern Australian,Circular Quay,Sydney,4.9,451,No
...,...,...,...,...,...,...,...,...
38,16600326,Mary Street Bakery,Bakery,Highgate,Perth,4.0,518,No
78,16601688,Tropico,"Tapas, Modern Australian, Pizza",North Beach,Perth,4.0,302,No
58,16599918,Jamie's Italian,Italian,Perth CBD,Perth,3.8,1473,No
89,16598361,The Wild Fig Cafe,"Modern Australian, Cafe Food",Scarborough,Perth,3.3,709,No


In [13]:
#get top restaurant per city
perth_top = restaurants_df.loc[restaurants_df["City"]=="Perth"]
perth_top = perth_top.head()
perth_top

,Restaurant ID,Name,Cuisine,Locality,City,Rating,Total Number Ratings,Delivery
71,16598976,Marumo,"Japanese, Seafood, Modern Australian",Nedlands,Perth,4.9,141,No
41,16598837,Run Amuk,Fast Food,"Orient Street, South Fremantle",Perth,4.9,341,No
8,16597513,Pacific Rim Mix Plate,"Hawaiian, Japanese",Applecross,Perth,4.9,213,No
60,16596002,Galileo Buona Cucina,Italian,Shenton Park,Perth,4.8,207,No
23,16598168,Nobu Perth,"Japanese, Sushi",Burswood,Perth,4.8,663,No
